# Student Groups Optimization

Use linear programming to assign students to groups subject to constraints specifying:
- __INDIVIDUALS__
    - __with:__ students to assign to a group together
    - __not with:__ students to assign to different groups
    - __in:__ students to assign to a specific group
    - __not in:__ students not to assign to a specific group
- __CHARACTERISTICS__ _--> advanced, disruptive, extrovert, IEP, introvert, male/female_
    - __homogenous:__ groups to assign only students with a specified characteristic
    - __maximum:__ maximum number of students with specified characteristic per group

## Enter Your Information

In [1]:
# Enter file path for Excel spreadsheet with student, group, and constraint data as "data/your_filename_here.xlsx"
# Example: "data/data_template.xlsx"
FILENAME = "data/data_template.xlsx"


# Enter desired number of optimal solutions
# Example: 3
NUM_SOLUTIONS = 3

## _Optional: Display student, group, and constraint data_

In [2]:
import pandas as pd

from src.assign_groups import *


print("STUDENT DATA...")
person_df = import_from_template(FILENAME, "Person Setup")
display(person_df)

print("GROUP DATA...")
group_df = import_from_template(FILENAME, "Grouping Setup")
display(group_df)

print("CONSTRAINT DATA...\n")

print("with...")
with_df = import_from_template(FILENAME, "Constraint - With")
display(with_df.head())

print("not with...")
not_with_df = import_from_template(FILENAME, "Constraint - Not With")
display(not_with_df.head())

print("in...")
in_df = import_from_template(FILENAME, "Constraint - In")
display(in_df.head())

print("not in...")
not_in_df = import_from_template(FILENAME, "Constraint - Not In")
display(not_in_df.head())

print("homogenous...")
hom_df = import_from_template(FILENAME,"Constraint - Homogenous")
display(hom_df.head())

print("max...")
max_df = import_from_template(FILENAME, "Constraint - Maximum")
display(max_df.head())

STUDENT DATA...


,name,advanced,disruptive,extrovert,iep,introvert,male
0,Ava,0,0,0,0,1,0
1,Brooklyn,0,0,0,0,1,0
2,Chloe,0,0,0,0,1,0
3,Faith,0,0,1,0,0,0
4,Grace,0,0,0,0,0,0
5,Hannah,0,0,0,0,0,0
6,Isabelle,0,0,0,0,0,0
7,Julia,1,0,0,0,1,0
8,Kaylee,1,0,0,0,1,0
9,Lily,0,0,0,1,0,0


GROUP DATA...


,group id,size
0,1,5
1,2,4
2,3,4
3,4,5


CONSTRAINT DATA...

with...


...

,name,group id
0,Ulysses,4


homogenous...


,group id,characteristic,value
0,3,iep,1


max...


,maximum,characteristic,value
0,2,introvert,1
1,2,extrovert,1


## Determine Optimal Groupings

In [3]:
import pandas as pd

from src.assign_groups import *


unique_solutions = []
person_df = import_from_template(FILENAME, "Person Setup")

for i in range(NUM_SOLUTIONS):

    print("Solving with rigid maximum constraint...")
    status, solution_df = run_lp_problem(FILENAME,
                                         elastic=False, 
                                         unique_solutions=unique_solutions)

    if status != "Optimal":
        
        print(f"{status}...")
        
        print("Solving with elastic maximum constraint...")
        status, solution_df = run_lp_problem(FILENAME,
                                             elastic=True,
                                             unique_solutions=unique_solutions)
        
        if status != "Optimal":
            print(f"Unsolvable with rigid or elastic constraint. Solution {status}.")
            break

    print(f"{status}...")
    
    # save unique optimal solution
    unique_solutions.append(solution_df)
    
    # display result
    print(f"\nSOLUTION {i}\n")
    solution_df = pd.merge(person_df, solution_df, on="name")
    for name, group in solution_df.groupby("group"):
        display(group)

Solving with rigid maximum constraint...
Optimal...

SOLUTION 0



,name,advanced,disruptive,extrovert,iep,introvert,male,group
4,Grace,0,0,0,0,0,0,1
6,Isabelle,0,0,0,0,0,0,1
7,Julia,1,0,0,0,1,0,1
10,Oscar,0,0,1,0,0,1,1
17,Vincent,0,1,0,0,1,1,1


,name,advanced,disruptive,extrovert,iep,introvert,male,group
0,Ava,0,0,0,0,1,0,2
2,Chloe,0,0,0,0,1,0,2
3,Faith,0,0,1,0,0,0,2
16,Ulysses,0,1,1,0,0,1,2


,name,advanced,disruptive,extrovert,iep,introvert,male,group
9,Lily,0,0,0,1,0,0,3
11,Paul,0,0,0,1,0,1,3
12,Quentin,0,0,0,1,0,1,3
13,Robert,0,0,0,1,0,1,3


,name,advanced,disruptive,extrovert,iep,introvert,male,group
1,Brooklyn,0,0,0,0,1,0,4
5,Hannah,0,0,0,0,0,0,4
8,Kaylee,1,0,0,0,1,0,4
14,Steven,0,1,1,0,0,1,4
15,Thomas,0,1,1,0,0,1,4


...

Solving with rigid maximum constraint...
Optimal...

SOLUTION 2



,name,advanced,disruptive,extrovert,iep,introvert,male,group
0,Ava,0,0,0,0,1,0,1
3,Faith,0,0,1,0,0,0,1
4,Grace,0,0,0,0,0,0,1
7,Julia,1,0,0,0,1,0,1
10,Oscar,0,0,1,0,0,1,1


,name,advanced,disruptive,extrovert,iep,introvert,male,group
1,Brooklyn,0,0,0,0,1,0,2
15,Thomas,0,1,1,0,0,1,2
16,Ulysses,0,1,1,0,0,1,2
17,Vincent,0,1,0,0,1,1,2


,name,advanced,disruptive,extrovert,iep,introvert,male,group
9,Lily,0,0,0,1,0,0,3
11,Paul,0,0,0,1,0,1,3
12,Quentin,0,0,0,1,0,1,3
13,Robert,0,0,0,1,0,1,3


,name,advanced,disruptive,extrovert,iep,introvert,male,group
2,Chloe,0,0,0,0,1,0,4
5,Hannah,0,0,0,0,0,0,4
6,Isabelle,0,0,0,0,0,0,4
8,Kaylee,1,0,0,0,1,0,4
14,Steven,0,1,1,0,0,1,4


## _Optional: Check constraints satisfied_
_Double check that the student grouping solution(s) displayed above satisfy your specified constraints._

In [4]:
# Enter True to display a DataFrame with your specified constraint data and False otherwise
verbose = False

In [5]:
from src.test_groups import *


person_df = import_from_template(FILENAME, "Person Setup")
with_df = import_from_template(FILENAME, "Constraint - With")
not_with_df = import_from_template(FILENAME, "Constraint - Not With")
in_df = import_from_template(FILENAME, "Constraint - In")
not_in_df = import_from_template(FILENAME, "Constraint - Not In")
hom_df = import_from_template(FILENAME,"Constraint - Homogenous")
max_df = import_from_template(FILENAME, "Constraint - Maximum")

for idx, solution_df in enumerate(unique_solutions):
    print(f"\nSOLUTION {idx}\n")
    df = pd.merge(person_df, solution_df, on="name")

    print("WITH...")
    test_with_constraint(with_df, df, verbose)

    print("\nNOT WITH...")
    test_not_with_constraint(not_with_df, df, verbose)

    print("\nIN...")
    test_in_constraint(in_df, df, verbose)

    print("\nNOT IN...")
    test_not_in_constraint(not_in_df, df, verbose)

    print("\nHOMOGENOUS...")
    test_homogenous_constraint(hom_df, df, verbose)

    print("\nMAXIMUM...")
    test_maximum_constraint(max_df, df, verbose)
    
    print()


SOLUTION 0

WITH...
	index 0: ok

NOT WITH...
	index 0: ok

IN...
	index 0: ok

NOT IN...
	index 0: ok

HOMOGENOUS...
	index 0: ok

MAXIMUM...
	index 0:
		group 1: ok (2)
		group 2: ok (2)
		group 3: ok (0)
		group 4: ok (2)
	index 1:
		group 1: ok (1)
		group 2: ok (2)
		group 3: ok (0)
		group 4: ok (2)


SOLUTION 1

WITH...
	index 0: ok

NOT WITH...
	index 0: ok

IN...
	index 0: ok

NOT IN...
	index 0: ok

HOMOGENOUS...
	index 0: ok

MAXIMUM...
	index 0:
		group 1: ok (2)
		group 2: ok (2)
		group 3: ok (0)
		group 4: ok (2)
	index 1:
		group 1: ok (2)
		group 2: ok (1)
		group 3: ok (0)
		group 4: ok (2)


SOLUTION 2

WITH...
	index 0: ok

NOT WITH...
	index 0: ok

IN...
	index 0: ok

NOT IN...
	index 0: ok

HOMOGENOUS...
	index 0: ok

MAXIMUM...
	index 0:
		group 1: ok (2)
		group 2: ok (2)
		group 3: ok (0)
		group 4: ok (2)
	index 1:
		group 1: ok (2)
		group 2: ok (2)
		group 3: ok (0)
		group 4: ok (1)



## _Optional: Check uniqueness of multiple solutions_

_Review the number of students who change groups from one solution to the next._

In [6]:
# Enter True to display a DataFrame with the names of students in each group and False otherwise
verbose = False

In [7]:
from src.test_groups import *

print("\n\nCHANGES BY SOLUTION...\n")
view_changes_by_solution(FILENAME, unique_solutions, verbose)

print("\n\nCHANGES BY GROUP...\n")
view_changes_by_group(FILENAME, unique_solutions, verbose)



CHANGES BY SOLUTION...


SOLUTION 0 --> SOLUTION 1
	 group 1: 4
	 group 2: 3
	 group 3: 0
	 group 4: 5
 ==> 12 total (66%)

SOLUTION 1 --> SOLUTION 2
	 group 1: 4
	 group 2: 2
	 group 3: 0
	 group 4: 5
 ==> 11 total (61%)


CHANGES BY GROUP...


GROUP 1
	solution 0 --> solution 1: 4 of 5
	solution 1 --> solution 2: 4 of 5

GROUP 2
	solution 0 --> solution 1: 3 of 4
	solution 1 --> solution 2: 2 of 4

GROUP 3
	solution 0 --> solution 1: 0 of 4
	solution 1 --> solution 2: 0 of 4

GROUP 4
	solution 0 --> solution 1: 5 of 5
	solution 1 --> solution 2: 5 of 5


## _Optional: Save results_

_Save student grouping solution(s) to Excel spreadsheet._

In [8]:
import datetime


# output filename as "groupings_yyyymmdd_hhmmss.xlsx"
output_filename = "results/groupings_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+".xlsx"

with pd.ExcelWriter(output_filename) as writer:
    for idx, solution_df in enumerate(unique_solutions):
        solution_df.sort_values("group").to_excel(writer, sheet_name=f"Grouping_{idx}", index=False)

print(f"Solution saved as \"{output_filename}\"")

Solution saved as "results/groupings_20191126_103851.xlsx"
